In [9]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://localhost:5000")

client = MlflowClient()

model = client.list_registered_models()

if not model:
    print("No registered models found.")
else:
    for m in model:
        print(f"Model name: {m.name}")
        for version in m.latest_versions:
            print(f" - Version: {version.version}, Stage: {version.current_stage}, Run ID: {version.run_id}")



AttributeError: 'MlflowClient' object has no attribute 'list_registered_models'

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient()

exp = client.get_experiment_by_name("Fever Detection Experiment")
if exp is None:
    print("Experiment not found.")
else:
    exp_id = exp.experiment_id
    runs = client.search_runs(
        experiment_ids=[exp_id],
        filter_string="",
        run_view_type=1,  # active runs; use 0 for all including deleted
        max_results=100,
        order_by=["attributes.start_time DESC"]
    )
    for r in runs:
        rid = r.info.run_id
        name = r.info.run_name
        status = r.info.status
        mt = r.data.params.get("model_types") or r.data.tags.get("model_types") or "<no model_types param>"
        print(f"{rid[:8]}  {name:20}  status={status:10}  model_types={mt}  metrics={r.data.metrics}")


Experiment not found.


In [ ]:
from mlflow.tracking import MlflowClient
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient()
experiments = client.list_experiments()
print("Experiments known to tracking server:")
for e in experiments:
    print(e.experiment_id, e.name, "lifecycle_stage=", e.lifecycle_stage)


AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

In [ ]:
#Training Pipeline Code
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pandas as pd
import numpy as np
import json
import os
import yaml
import logging
from datetime import datetime 
import mlflow
from .data_processor import get_data_preprocessor
from .model import Model
from mlflow.tracking import MlflowClient



logger = logging.getLogger(__name__)
class TrainingPipeline:
    def __init__(self, preprocessor, model_types=None, random_state=None, **model_params):

        param_file_path = "C:/Users/DELL/Desktop/my_ml_project/notebook/params.yaml"

        try:
            with open(param_file_path, 'r') as f:
                params = yaml.safe_load(f)


            self.model_types = model_types or params['model']['model_types']
            self.random_state = random_state or params['model']["random_forest", "logistic_regression", "xgboost", "svm", "knn", "decision_tree"]
            self.model_params = model_params or params['model'].get(self.model_types, {})
            
        except Exception as e:
            print(f"⚠️  Could not load params.yaml: {e}. Using defaults.")
            self.model_types = model_types or ''
            self.random_state = random_state or 42
            self.model_params = model_params or {}

        #try:
        #    #mlflow.sklearn.autolog()
        #    self.mlflow_autolog_enabled = True  # Specific to autologging
        #    logger.info("MLflow autologging enabled successfully")
        #except Exception as e:
        #    self.mlflow_autolog_enabled = False
        #    logger.warning(f"MLflow autologging failed: {str(e)}. Manual logging still available.")
        try:
            mlflow.sklearn.autolog(disable=True)  # Disable here too
        except:
            pass

        self.preprocessor = preprocessor
        self.model = Model(
            model_types=self.model_types, 
            random_state=self.random_state, 
            **self.model_params
        )
        self.pipeline = None
        self.training_history = {}
        self.evaluation_results = {}
        self._create_pipeline()

    def _create_pipeline(self):
        """
        create the pipeline with preprocessor and classifier
        """
        if self.model.classifier is None:
            raise ValueError("Model classifier is not initialised")
        
        self.pipeline = Pipeline(steps=[
            ('preprocessor', self.preprocessor),
            ('classifier', self.model.classifier)
        ])
        print(f"✅ Created pipeline with {self.model_types} classifier")

    def fit(self, X, y, experiment_name= "Fever Dectection Experiment", 
            run_name=None):
        """
        Train the model and track training history
        """
        if self.pipeline is None:
            self._create_pipeline()

        # ensure experiment exists
        mlflow.set_tracking_uri("http://localhost:5000")
        mlflow.set_experiment(experiment_name)
        start_time = datetime.now()
        
        if run_name is None:
            run_name = f"{self.model_types}_run_{int(datetime.now().strftime('%Y%m%d_%H%M%S'))}"

        #mlflow.set_tracking_uri("http://localhost:5000")

        with mlflow.start_run(run_name=run_name) as run:
            
            run_id = run.info.run_id
            self.training_history["mlflow_run_id"] = run_id

            # Log model and training parameters
            mlflow.log_param("model_type", self.model_type)
            mlflow.log_params(self.model_params)

            #start_time = datetime.now()
     
            self.pipeline.fit(X, y)

            training_time = (datetime.now() - start_time).total_seconds()

            self.training_history = {
                'training_time_seconds': training_time,
                'training_samples': len(X),
                'features_count': X.shape[1],
                'model_types': self.model_types,
                'random_state': self.random_state,
            'training_timestamp': datetime.now().isoformat()
            }

            try:
                mlflow.log_metric("training_time", training_time)
                mlflow.log_param("feature_count", X.shape[1])
                mlflow.log_param("model_types", self.model_types)
                
                if hasattr(self, 'pipeline') and self.pipeline is not None:
                    mlflow.sklearn.log_model(self.pipeline, "model")
                    
            except Exception as e:
                logger.warning(f"Manual MLflow logging failed: {str(e)}")

            print(f"✅ Training completed in {training_time:.2f} seconds")

        return self

    def predict(self, X):
        """Make predictions"""
        if self.pipeline is None:
            raise ValueError("Model must be trained before making predictions")
        return self.pipeline.predict(X)
    
    def predict_proba(self, X):
        """Get prediction probabilities"""
        if self.pipeline is None:
            raise ValueError("Model must be trained before making predictions")
        return self.pipeline.predict_proba(X)

    def evaluate(self, X, y, dataset_name='validation'):
        """
        Comprehensive evaluation on a dataset
        
        Args:
            X: Features
            y: True labels
            dataset_name: Name of the dataset for reporting
            
        Returns:
            dict: Comprehensive evaluation metrics
        """
        if self.pipeline is None:
            raise ValueError("Model must be trained before evaluation")
        
        # Make predictions
        y_pred = self.predict(X)
        y_pred_proba = self.predict_proba(X)
        
        # metrics
        metrics = {
            'accuracy': accuracy_score(y, y_pred),
            'precision': precision_score(y, y_pred, average='weighted', zero_division=0),
            'recall': recall_score(y, y_pred, average='weighted', zero_division=0),
            'f1': f1_score(y, y_pred, average='weighted', zero_division=0),
        }
        
        # ROC-AUC 
        if len(np.unique(y)) == 2:
            metrics['roc_auc'] = roc_auc_score(y, y_pred_proba[:, 1])
        else:
            metrics['roc_auc'] = roc_auc_score(y, y_pred_proba, multi_class='ovr', average='weighted')
        
        cm = confusion_matrix(y, y_pred)
        metrics['confusion_matrix'] = cm.tolist()
        
        metrics['class_distribution'] = {
            'class_0': int((y == 0).sum()),
            'class_1': int((y == 1).sum()),
            'class_0_percentage': float((y == 0).mean() * 100),
            'class_1_percentage': float((y == 1).mean() * 100)
        }
        
        self.evaluation_results[dataset_name] = metrics
        
        return metrics
    
    def get_comprehensive_report(self, X_train, y_train, X_val, y_val):
        """
        Generate comprehensive training and validation report
        
        Args:
            X_train: Training features
            y_train: Training labels
            X_val: Validation features
            y_val: Validation labels
            
        Returns:
            dict: Comprehensive report with training and validation metrics
        """
        
        train_metrics = self.evaluate(X_train, y_train, 'training')
        val_metrics = self.evaluate(X_val, y_val, 'validation')
        
        # Calculate overfitting indicators
        accuracy_gap = train_metrics['accuracy'] - val_metrics['accuracy']
        f1_gap = train_metrics['f1'] - val_metrics['f1']

        # Compute performance gap
        perf_gap = {
            metric: abs(train_metrics.get(metric, 0) - val_metrics.get(metric, 0))
            for metric in ["accuracy", "precision", "recall", "f1"]
        }
        
        generalization_quality = "good"
        if perf_gap["accuracy"] > 0.1:
            generalization_quality = "possible_overfit"

        comprehensive_report = {
            'training_metadata': self.training_history,
            'training_metrics': train_metrics,
            'validation_metrics': val_metrics,
            'performance_analysis': {
                'accuracy_gap': accuracy_gap,
                'f1_gap': f1_gap,
                'is_overfitting': accuracy_gap > 0.1,  # More than 10% gap
                'is_underfitting': train_metrics['accuracy'] < 0.7,  # Poor training performance
                'generalization_quality': 'good' if accuracy_gap < 0.05 else 'moderate' if accuracy_gap < 0.1 else 'poor'
            },
            'model_summary': {
                'model_type': self.model_type,
                'best_metric': 'accuracy',
                'best_score': val_metrics['accuracy'],
                'training_status': 'completed'
            }
        }
        # Log model performance characteristics
        run_id = self.training_history.get("mlflow_run_id")
        if run_id:
            with mlflow.start_run(run_id=run_id):
                mlflow.set_tag("generalization_quality", comprehensive_report['performance_analysis']['generalization_quality'])
                mlflow.set_tag("is_overfitting", str(comprehensive_report['performance_analysis']['is_overfitting']))

        return comprehensive_report
    
    def save_training_report(self, report, filepath="reports/training_metrics.json"):
        """Save training report to JSON file"""
        import json
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        # Convert numpy types to Python native types for JSON serialization
        def convert_to_serializable(obj):
            if isinstance(obj, (np.integer, np.int64, np.int32)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float64, np.float32)):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, dict):
                return {key: convert_to_serializable(value) for key, value in obj.items()}
            elif isinstance(obj, (list, tuple)):
                return [convert_to_serializable(item) for item in obj]
            else:
                return obj
        
        serializable_report = convert_to_serializable(report)
        
        with open(filepath, 'w') as f:
            json.dump(serializable_report, f, indent=2)
        
        print(f"✅ Training report saved to {filepath}")
        run_id = self.training_history.get("mlflow_run_id")
        if run_id:
            with mlflow.start_run(run_id=run_id):
                mlflow.log_artifact(filepath)
        return filepath
    
    def save_model(self, filepath):
        """Save the trained pipeline"""
        if self.pipeline is None:
            raise ValueError("No trained model to save")
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        # Save using the model's save method
        self.model.pipeline = self.pipeline
        self.model.save(filepath)
        
        print(f"✅ Model saved to {filepath}")


    # Register the model in MLflow Model Registry
    def register_model(self, model_name, transition_to_stage="Staging"):
        """
        Register the model in MLflow Model Registry.
        """
        run_id = self.training_history.get("mlflow_run_id")
        if not run_id:
            raise ValueError("No MLflow run ID found. Train the model first using .fit().")

        client = MlflowClient()
        model_uri = f"runs:/{run_id}/model"

        # Create or update model version
        model_version = client.create_model_version(
            name=model_name,
            source=model_uri,
            run_id=run_id
        )

        print(f"📦 Model version {model_version.version} registered under '{model_name}'")

        if transition_to_stage:
            client.transition_model_version_stage(
                name=model_name,
                version=model_version.version,
                stage=transition_to_stage
            )
            print(f"🚀 Transitioned '{model_name}' v{model_version.version} to stage: {transition_to_stage}")

        return model_version
    
    def load_model(self, filepath):
        """Load a trained pipeline"""
        self.model.load(filepath)
        self.pipeline = self.model.pipeline
        print(f"✅ Model loaded from {filepath}")

ImportError: attempted relative import with no known parent package

In [ ]:
#Model.py
# src/ml/model.py
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')
import yaml

XGBOOST_AVAILABLE = True

class Model:
    def __init__(self, model_types= ["random_forest", "logistic_regression", "xgboost", "svm", "knn", "decision_tree"], random_state=42, **model_params):
        """
        Initialize model with specified type and parameters
        
        Args:
            model_type (str): Type of model to use
            random_state (int): Random seed for reproducibility
            **model_params: Additional model-specific parameters
        """
        self.pipeline = None
        self.model_types = model_types
        self.random_state = random_state
        self.model_params = model_params
        self.classifier = None
        self.initialise_model()
    
    def initialise_model(self):
        """Initialize the classifier based on model type using parameters from params.yaml"""
        param_path_file = "C:/Users/DELL/Desktop/my_ml_project/notebook/params.yaml"

        try:
            with open(param_path_file, 'r') as f:
                params = yaml.safe_load(f)
            model_params = params['model']
        except:
            # Fallback to default parameters if params.yaml not available
            model_params = {}
        
        if self.model_types == 'random_forest':
            rf_params = model_params.get('random_forest', {})
            self.classifier = RandomForestClassifier(
                n_estimators=rf_params.get('n_estimators', self.model_params.get('n_estimators', 100)),
                max_depth=rf_params.get('max_depth', self.model_params.get('max_depth', None)),
                min_samples_split=rf_params.get('min_samples_split', self.model_params.get('min_samples_split', 2)),
                min_samples_leaf=rf_params.get('min_samples_leaf', self.model_params.get('min_samples_leaf', 1)),
                random_state=self.random_state,
                n_jobs=-1
            )
            
        elif self.model_types == 'logistic_regression':
            lr_params = model_params.get('logistic_regression', {})
            self.classifier = LogisticRegression(
                C=lr_params.get('C', self.model_params.get('C', 1.0)),
                penalty=lr_params.get('penalty', self.model_params.get('penalty', 'l2')),
                solver=lr_params.get('solver', self.model_params.get('solver', 'lbfgs')),
                max_iter=lr_params.get('max_iter', self.model_params.get('max_iter', 1000)),
                random_state=self.random_state,
                n_jobs=-1
            )
            
        elif self.model_types == 'xgboost':
            xgb_params = model_params.get('xgboost', {})
            self.classifier = XGBClassifier(
                n_estimators=xgb_params.get('n_estimators', self.model_params.get('n_estimators', 100)),
                max_depth=xgb_params.get('max_depth', self.model_params.get('max_depth', 6)),
                learning_rate=xgb_params.get('learning_rate', self.model_params.get('learning_rate', 0.1)),
                subsample=xgb_params.get('subsample', self.model_params.get('subsample', 1.0)),
                colsample_bytree=xgb_params.get('colsample_bytree', self.model_params.get('colsample_bytree', 1.0)),
                random_state=self.random_state,
                n_jobs=-1,
                eval_metric='logloss'
            )
            
        elif self.model_types == 'svm':
            svm_params = model_params.get('svm', {})
            self.classifier = SVC(
                C=svm_params.get('C', self.model_params.get('C', 1.0)),
                kernel=svm_params.get('kernel', self.model_params.get('kernel', 'rbf')),
                probability=True,
                random_state=self.random_state
            )
            
        elif self.model_types == 'knn':
            knn_params = model_params.get('knn', {})
            self.classifier = KNeighborsClassifier(
                n_neighbors=knn_params.get('n_neighbors', self.model_params.get('n_neighbors', 5)),
                weights=knn_params.get('weights', self.model_params.get('weights', 'uniform')),
                n_jobs=-1
            )
            
        elif self.model_types == 'decision_tree':
            dt_params = model_params.get('decision_tree', {})
            self.classifier = DecisionTreeClassifier(
                max_depth=dt_params.get('max_depth', self.model_params.get('max_depth', None)),
                min_samples_split=dt_params.get('min_samples_split', self.model_params.get('min_samples_split', 2)),
                min_samples_leaf=dt_params.get('min_samples_leaf', self.model_params.get('min_samples_leaf', 1)),
                random_state=self.random_state
            )
            
        else:
            available_models = ['random_forest', 'logistic_regression', 'xgboost', 'svm', 'knn', 'decision_tree']
            raise ValueError(f"Model type '{self.model_types}' not supported. Available models: {available_models}")
        
        print(f"✅ Initialized {self.model_types} classifier with parameters from params.yaml")
        
    def get_model_info(self):
        """Get information about the current model"""
        if self.classifier is None:
            return {"status": "Model not initialized"}
        
        info = {
            "model_types": self.model_types,
            "classifier": str(self.classifier.__class__.__name__),
            "parameters": self.classifier.get_params(),
            "is_fitted": hasattr(self.classifier, 'classes_')
        }
        return info
    
    def fit(self, X, y):
        """Fit the model to training data"""
        if self.pipeline is not None:
            self.pipeline.fit(X, y)
        elif self.classifier is not None:
            self.classifier.fit(X, y)
        else:
            raise ValueError("No model initialized. Call initialise_model() first.")
    
    def predict(self, X):
        """Make predictions"""
        if self.pipeline is not None:
            return self.pipeline.predict(X)
        elif self.classifier is not None:
            return self.classifier.predict(X)
        else:
            raise ValueError("No model available for prediction")
    
    def predict_proba(self, X):
        """Get prediction probabilities"""
        if self.pipeline is not None:
            return self.pipeline.predict_proba(X)
        elif self.classifier is not None:
            # Check if classifier supports predict_proba
            if hasattr(self.classifier, 'predict_proba'):
                return self.classifier.predict_proba(X)
            else:
                raise ValueError(f"{self.model_type} does not support probability predictions")
        else:
            raise ValueError("No model available for prediction")
    
    def score(self, X, y):
        """Return the accuracy score"""
        if self.pipeline is not None:
            return self.pipeline.score(X, y)
        elif self.classifier is not None:
            return self.classifier.score(X, y)
        else:
            raise ValueError("No model available for scoring")
    
    def save(self, filepath):
        """Save the model/pipeline to file"""
        if self.pipeline is not None:
            joblib.dump(self.pipeline, filepath)
        elif self.classifier is not None:
            joblib.dump(self.classifier, filepath)
        else:
            raise ValueError("No model to save")
    
    def load(self, filepath):
        """Load a model/pipeline from file"""
        loaded_obj = joblib.load(filepath)
        
        # Determine if it's a pipeline or classifier
        if hasattr(loaded_obj, 'steps'):  # It's a pipeline
            self.pipeline = loaded_obj
            # Extract classifier from pipeline
            for name, step in loaded_obj.steps:
                if name == 'classifier':
                    self.classifier = step
                    break
        else:  # It's a classifier
            self.classifier = loaded_obj
        
        # Infer model type from the loaded object
        self._infer_model_type(loaded_obj)
    
    def _infer_model_type(self, model_obj):
        """Infer model type from loaded object"""
        model_class = model_obj.__class__.__name__
        
        if 'RandomForest' in model_class:
            self.model_types = 'random_forest'
        elif 'LogisticRegression' in model_class:
            self.model_types = 'logistic_regression'
        elif 'XGB' in model_class:
            self.model_types = 'xgboost'
        elif 'SVC' in model_class:
            self.model_types = 'svm'
        elif 'KNeighbors' in model_class:
            self.model_types = 'knn'
        elif 'DecisionTree' in model_class:
            self.model_types = 'decision_tree'
        else:
            self.model_types = 'unknown'

In [11]:
import mlflow
from src.ml.training_pipeline import TrainingPipeline
from src.ml.data_processor import get_data_preprocessor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 1. Create dataset
X, y = make_classification(n_samples=500, n_features=10, n_classes=2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Preprocessor
preprocessor = get_data_preprocessor()

# 3. Define experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Fever Detection Experiment")

# 4. Loop over all models
model_types = ["random_forest", "logistic_regression", "xgboost", "svm", "knn", "decision_tree"]

for model_type in model_types:
    print(f"--- Training {model_type} ---")

    # Load model-specific params safely from YAML or defaults
    # Example: you could load from your YAML here, but keep only valid keys
    # For demo, we use empty dict (defaults)
    raw_params = {}  # Replace with YAML params if needed
    filtered_params = filter_valid_params(MODEL_CLASS_MAP[model_type], raw_params)

    # Initialize pipeline
    tp = TrainingPipeline(preprocessor=preprocessor, model_type=model_type, model_params=filtered_params)

    # Train & log to MLflow
    tp.fit(X_train, y_train, experiment_name="Fever Detection Experiment", run_name=f"{model_type}_run")

    # Evaluate and save report
    report = tp.get_comprehensive_report(X_train, y_train, X_val, y_val)
    tp.save_training_report(report, filepath=f"reports/{model_type}_metrics.json")

    # Register model in MLflow
    tp.register_model(model_name="FeverSeverityModel", transition_to_stage="Staging")


ModuleNotFoundError: No module named 'src'

In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/DELL/Desktop/my_ml_project/notebook/fever.csv")   # or the CSV you use as raw input
print("columns:", df.columns.tolist())
print("target dtype:", df["Fever_Severity_code"].dtype)
print("unique values & counts:\n", df["Fever_Severity_code"].value_counts())
print(df[["Fever_Severity_code"]].head())


columns: ['Temperature', 'Fever_Severity', 'Age', 'Gender', 'BMI', 'Headache', 'Body_Ache', 'Fatigue', 'Chronic_Conditions', 'Allergies', 'Smoking_History', 'Alcohol_Consumption', 'Humidity', 'AQI', 'Physical_Activity', 'Diet_Type', 'Heart_Rate', 'Blood_Pressure', 'Previous_Medication', 'Recommended_Medication']


KeyError: 'Fever_Severity_code'

In [3]:
from pathlib import Path
params_path = Path("C:/Users/DELL/Desktop/my_ml_project/notebook/params.yaml")
print(params_path.exists())


True


In [ ]:
# find_experiment.py
from data_processor import load_params

params = load_params()
print("🔍 Checking where experiment_name is in your params.yaml...")

# Check all possible locations
locations = [
    ('training.experiment_name', params.get('training', {}).get('experiment_name')),
    ('train.experiment_name', params.get('train', {}).get('experiment_name')), 
    ('experiment_name (root)', params.get('experiment_name'))
]

for location, value in locations:
    if value:
        print(f"✅ FOUND: {location} = '{value}'")
    else:
        print(f"❌ NOT FOUND: {location}")

print(f"\n🎯 Your experiment name is: '{params.get('training', {}).get('experiment_name')}'")
print("   Look for THIS exact name in MLflow UI")

ImportError: attempted relative import with no known parent package

In [ ]:
class Config:
        schema_extra = {
            "example": {
                "temperature": 38.5,
                "age": 35,
                "bmi": 24.2,
                "humidity": 65.0,
                "aqi": 45.0,
                "heart_rate": 85.0,
                "gender": "Male",
                "headache": "Yes",
                "body_ache": "No",
                "fatigue": "Yes",
                "chronic_conditions": "None",
                "allergies": "No",
                "smoking_history": "Non-smoker",
                "alcohol_consumption": "Occasional",
                "physical_activity": "Moderate",
                "diet_type": "Balanced",
                "blood_pressure": "Normal",
                "previous_medication": "None",
                "recommended_medication": "Paracetamol"
            }
        }

class Config:
        schema_extra = {
            "example": {
                "prediction": 1,
                "probability": 0.85,
                "risk_level": "High"
            }
        }

class Config:
        schema_extra = {
            "example": {
                "message": "FeverSeverity Prediction API is running",
                "status": "success"
            }
        }

class Config:
        schema_extra = {
            "example": {
                "status": "healthy",
                "message": "API is running normally"
            }
        }

In [ ]:
from fastapi import FastAPI, HTTPException
import sys
import os

# Add src to path to import your ML modules
sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

from src.ml.model_loader import load_production_model
from api.models import PatientData, PredictionResponse, HealthResponse, APIResponse

app = FastAPI(
    title="FeverSeverity Prediction API", 
    version="1.00",
    description="API for predicting fever severity based on patient data",
    docs_url="/docs",
    redoc_url="/redoc"
)

# Load your production model
try:
    model = load_production_model()
    print("✅ Model loaded successfully")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    model = None

@app.get("/", response_model=APIResponse)
async def root():
    return APIResponse(message="FeverSeverity Prediction API is running")

@app.get("/health", response_model=HealthResponse)
async def health_check():
    if model is None:
        return HealthResponse(status="unhealthy", message="Model not loaded")
    return HealthResponse(status="healthy", message="API and model are ready")

@app.post("/predict", response_model=PredictionResponse)
async def predict(patient_data: PatientData):
    # Check if model is loaded
    if model is None:
        raise HTTPException(status_code=503, detail="Model not loaded")
    
    try:
        # Convert patient data to features array
        features = [
            patient_data.temperature,
            patient_data.age,
            patient_data.bmi,
            patient_data.humidity,
            patient_data.aqi,
            patient_data.heart_rate,
            patient_data.gender,
            patient_data.headache,  
            patient_data.body_ache,
            patient_data.fatigue,
            patient_data.chronic_conditions,
            patient_data.allergies,
            patient_data.smoking_history,
            patient_data.alcohol_consumption,
            patient_data.physical_activity,
            patient_data.diet_type,
            patient_data.blood_pressure,
            patient_data.previous_medication,
            patient_data.recommended_medication
        ]
        
        # Make prediction
        prediction = model.predict([features])[0]
        probability = model.predict_proba([features])[0][1]  # Probability of positive class

        # Determine the risk level
        risk_level = "High" if prediction == 1 else "Low"

        return PredictionResponse(
            prediction=int(prediction),
            probability=float(probability),
            risk_level=risk_level
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


